In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import forecast
from quote_chart import create_chart_app

start_date = '2024-01-02 00:00' # default start date.
selected_period = '1T' # default period. allowed values are as periods in pandas.resample. for example, 1T == 1 minute.
# load data for the default time period.
candles_df = forecast.get_df('BTCUSD', start_date, selected_period)


def on_period_change(button_id):
    global candles_df, selected_period
    if button_id == '':
        return
    selected_period = button_id
    candles_df = forecast.get_df('BTCUSD', start_date, selected_period)


def create_figure(x0, x1):
    #print('create_figure started')
    # slice dataframe so that there will be enough data to plot the chart and also have data on the left and right so 
    # that when the user starts to zoom/pan he will see data.
    if x0 is not None:
        delta = x1 - x0
        df = candles_df[x0-delta:x1+delta]
    else:
        # by default show last 100 candles.
        df = candles_df[-100:]
    # define multiple panes. The top pane will be for the main price chart with candles. The second pane is for volumes.
    fig = make_subplots(rows=3, cols=1, shared_xaxes=True, 
                vertical_spacing=0.01,
                row_heights=[0.8, 0.2, 0.2],
                specs=[[{"secondary_y": True}], [{"secondary_y": True}], [{"secondary_y": True}]])
    fig.add_trace(go.Scatter(x=df.index, y=df['high'] * 1.001, mode='lines', line=dict(color='red'), name='highs'), row=3, col=1)
    fig.add_trace(go.Scatter(x=df.index, y=df['low'] * 0.999, mode='lines', line=dict(color='steelblue'), name='lows'), row=3, col=1)
    # plot the main chart with price candles.
    fig.add_trace(go.Candlestick(x=df.index,
                                 open=df['open'],
                                 high=df['high'],
                                 low=df['low'],
                                 close=df['close'],
                                 name=selected_period), row=1, col=1)
    fig.add_trace(go.Bar(x=df.index, y=df['volume'], name='Volume'), row=2, col=1)
    # set the default dragmode to pan, remove the range slider because i use zoom/pan instead of it.
    fig.update_layout(
        dragmode='pan',
        xaxis_rangeslider_visible=False,
        width=1200, # px
        height=600,
        margin=dict(l=3, r=0, t=0, b=0),
        yaxis=dict(side='right'),
        # yaxis2=dict(side='right'),
        yaxis3=dict(side='right'),
        # yaxis4=dict(side='right'),
        yaxis5=dict(side='right'),
        )
    fig.update_xaxes(
        ticklabelposition="outside right",  # keep labels on the right so that they don't affect margin-left.
    )
    #print('create_figure completed')
    return fig


app = create_chart_app(create_figure, on_period_change)
app.run_server(debug=True)